### 필요한 패키지 임포트

In [1]:
import os
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv
import time
import datetime

### 파일 경로 불러오기

In [2]:
# 파일 경로
general = "/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN"
genre = ('blues classic disco hiphop idol jazz metal pop rock trot').split(" ")

all_audio_path = []

for g in genre:
    audio_path = general + f'/{g}'
    all_audio_path.append(audio_path)

In [3]:
all_audio_path

['/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/blues',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/classic',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/disco',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/hiphop',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/idol',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/jazz',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/pop',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/rock',
 '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/trot']

### 함수 정의

In [4]:
def song_names(audio_path):                      # audio_path : 음원 파일들의 폴더 경로
    files = list(os.listdir(f'{audio_path}'))

    songs = []
    paths = []

    for file in files:
        song = f'{file}'
        path = os.path.join(audio_path, file)
        songs.append(song)
        paths.append(path)

    return songs, paths

In [5]:
def all_songs_paths(all_audio_path):

    all_song = []
    all_path = []

    for audio_path in all_audio_path:
        songs, paths = song_names(audio_path)
        all_song.append(songs)
        all_path.append(paths)

    return all_song, all_path

In [6]:
all_song, all_path = all_songs_paths(all_audio_path)

In [7]:
#all_song    # 모든 노래 파일 : [[],[],[],...,[]]

In [8]:
#all_path    # 모든 파일 경로 : [[],[],[],...,[]]

In [9]:
#all_path[0] # blues 파일 목록

In [10]:
#for path in all_path:
#    for p in path:
#        print(p)
#        print(p.split("/")[-1])

In [11]:
# soundwave
def soundwave(all_path):

    audio_files = []
    song_name = []

    for path in all_path:
        for p in path:
            try:
                sn = p.split("/")[-1]
                y, sr = librosa.load(p, duration = 30)
                y, _ = librosa.effects.trim(y)
                song_name.append(sn)
                audio_files.append(y)

            except ValueError as val:
                print(path, val)

    return audio_files, song_name, sr # sr 디폴트 : 22050 / 변경 가능한 sr 수치 : 44100

In [12]:
#audio_files, song_name, sr = soundwave(all_path)

In [13]:
#len(audio_files), len(song_name)

In [14]:
# Features
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # harmony, perceptral treatment (+@ 부분)
    harm, perc = librosa.effects.hpss(y)
    features['harm'] = harm.ravel()
    features['perc'] = perc.ravel()

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr)   # n_mfcc=20 (default) 
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
#    onset_env = librosa.onset.onset_strength(y=audio_file, sr=sr)
#    dict_agg_features['tempo'] = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [15]:
# Extract features 
def final_extraction(dataset_dir=all_audio_path):

    last_features = []

    all_song, all_path = all_songs_paths(all_audio_path)
    audio_files, song_name, sr = soundwave(all_path)

    for audio in audio_files:
        features = get_features(audio, sr)
        last_features.append(features)

    return last_features, song_name

In [ ]:
%%time

last_features, song_name = final_extraction(all_audio_path)

['/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00025.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00023.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00027.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00026.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00024.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00030.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00033.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00037.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/metal/metal.00041.wav', '/content/drive/Othercomputers/내 노트북/Project/Dataset/Genre10_Likes_GTZAN/met

In [20]:
len(last_features), len(song_name)

(1204, 1204)

In [23]:
def df_all_features(last_features, songs):
    df_features = pd.DataFrame(last_features)
    df_songs = pd.DataFrame(song_name, columns=['song'])

    return pd.concat([df_songs, df_features], axis=1)

In [24]:
df_all_features = df_all_features(last_features, song_name)
df_all_features.head()

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,blues.00021.wav,2755.434327,429.592861,1046.748840,540.054654,0.327042,1.129839,5189.501953,258.398438,1830.788929,...,10.856279,2.202138,1.095310,45.731815,-42.664536,-0.780010,12.189150,0.817654,0.300143,99.384014
1,blues.00022.wav,3469.748131,397.000759,1156.070496,560.664619,1.051891,1.124727,5673.999023,247.631836,2170.053545,...,10.924349,2.228990,1.204073,52.893822,-35.879383,-0.608485,11.974702,1.862670,0.749557,99.384014
2,blues.00024.wav,2851.339646,433.586661,1120.387619,420.021450,-0.050562,0.717949,5221.801758,269.165039,2325.102607,...,6.651611,0.143240,-0.305405,24.187828,-43.491180,-1.596130,9.138107,1.272281,-0.672218,103.359375
3,blues.00020.wav,3205.333473,687.159992,1389.955510,320.782562,4.333950,1.310306,6513.793945,645.996094,3005.773491,...,9.701139,1.884183,0.988186,33.259712,-22.959686,1.508026,8.592778,0.657548,0.346329,135.999178
4,blues.00025.wav,2304.512028,406.256758,1109.569627,372.486717,0.134566,0.764230,4974.169922,355.297852,2243.569767,...,8.717470,0.223954,-0.263630,29.286180,-30.642246,-0.533179,8.226832,0.332313,-0.034046,92.285156


In [25]:
# export the data
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
df_all_features.to_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv')

In [26]:
# import the data
df_all_features = pd.read_csv(f'/content/drive/Othercomputers/내 노트북/Project/Models/Music_Genre_Classification_Recommendation/Feature_Result_csv/{filename}.csv', index_col = 0)

In [27]:
df_all_features

,song,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,...,mfcc_18_std,mfcc_18_kurtosis,mfcc_18_skew,mfcc_19_max,mfcc_19_min,mfcc_19_mean,mfcc_19_std,mfcc_19_kurtosis,mfcc_19_skew,tempo
0,blues.00021.wav,2755.434327,429.592861,1046.748840,540.054654,0.327042,1.129839,5189.501953,258.398438,1830.788929,...,10.856279,2.202138,1.095310,45.731815,-42.664536,-0.780010,12.189150,0.817654,0.300143,99.384014
1,blues.00022.wav,3469.748131,397.000759,1156.070496,560.664619,1.051891,1.124727,5673.999023,247.631836,2170.053545,...,10.924349,2.228990,1.204073,52.893820,-35.879383,-0.608485,11.974702,1.862670,0.749557,99.384014
2,blues.00024.wav,2851.339646,433.586661,1120.387619,420.021450,-0.050562,0.717949,5221.801758,269.165039,2325.102607,...,6.651611,0.143240,-0.305405,24.187828,-43.491180,-1.596130,9.138107,1.272281,-0.672218,103.359375
3,blues.00020.wav,3205.333473,687.159992,1389.955510,320.782562,4.333950,1.310306,6513.793945,645.996094,3005.773491,...,9.701139,1.884183,0.988186,33.259712,-22.959686,1.508026,8.592778,0.657548,0.346329,135.999178
4,blues.00025.wav,2304.512028,406.256758,1109.569627,372.486717,0.134566,0.764230,4974.169922,355.297852,2243.569767,...,8.717470,0.223954,-0.263630,29.286180,-30.642246,-0.533179,8.226832,0.332313,-0.034046,92.285156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,항구의 남자-박상철_30.wav,3831.797847,1339.280765,2418.792343,427.316018,0.005328,0.438842,7310.522461,3079.248047,4610.730453,...,8.239953,-0.103871,0.145450,25.572388,-25.593060,0.788201,7.837722,0.085515,0.233073,135.999178
1200,호랑나비-김흥국_30.wav,6487.719679,739.011685,2689.284670,849.794691,2.621117,0.826447,9377.709961,872.094727,5580.916253,...,8.059708,3.347126,1.289246,34.164238,-25.492624,-0.965621,7.650224,0.965897,0.063840,135.999178
1201,10분 내로-김연자_30.wav,6896.682494,1443.729408,2949.338935,703.271173,5.844847,1.890425,9722.241211,2680.883789,6646.676475,...,9.628524,0.632032,0.436896,41.290916,-23.750210,3.242226,9.922538,0.505130,0.671343,143.554688
1202,사랑의 트위스트-설운도_30.wav,5206.455338,854.989721,2361.416274,520.588591,3.560736,0.544171,8613.281250,1324.291992,4815.887546,...,7.695136,1.393292,-0.680192,27.008787,-15.746796,4.046974,6.830245,0.075743,0.327824,161.499023


### 특정값 포함하는 데이터 추출 (데이터 분포 확인용)

In [51]:
contains_metal = df_all_features['song'].str.contains("metal")
subset_df = df_all_features[contains_metal]

In [57]:
subset_df['song'].count()

77